# Project 6 Lab Journal
Filippov M., Zheltova K

Data avaiabe at https://figshare.com/articles/dataset/_Dead_man_s_teeth_dataset/12152040

Original data: https://trace.ncbi.nlm.nih.gov/Traces/index.html?view=study&acc=SRP029257

### 1 QIIME (installed with conda, scripts here wont work)
Importing FASTQ files (manifest file)

`qiime tools import   --type 'SampleData[SequencesWithQuality]'   --input-path manifest.tsv   --output-path sequences.qza   --input-format SingleEndFastqManifestPhred33V2` 

`qiime tools validate sequences.qza`
> Result sequences.qza appears to be valid at level=max.

QC

`qiime demux summarize --i-data sequences.qza --o-visualization sequences.qzv`


Trim barcode (mostleft 35 bp) and amplification primer (140 bp)

`qiime dada2 denoise-single   --i-demultiplexed-seqs sequences.qza   --p-trim-left 35 --p-trunc-len 140 --o-representative-sequences rep-seqs.qza --o-table table.qza --o-denoising-stats stats.qza`

Create QC table

`qiime metadata tabulate   --m-input-file stats.qza   --o-visualization stats.qzv`

Visualization table

`qiime feature-table summarize   --i-table table.qza   --o-visualization table.qzv   --m-sample-metadata-file sample-metadata.tsv`

Map OTUs to sequences

`qiime feature-table tabulate-seqs  --i-data rep-seqs.qza   --o-visualization rep-seqs.qzv`

Download Silva DB here

https://docs.qiime2.org/2023.2/data-resources/#marker-gene-reference-databases

Download Naive Bayes classifier trained on this data

https://data.qiime2.org/2022.2/common/silva-138-99-nb-classifier.qza

Get taxonomy

`qiime feature-classifier classify-sklearn   --i-classifier silva-138-99-nb-classifier.qza   --i-reads rep-seqs.qza   --o-classification taxonomy.qza` # out of RAM, downoaded taxonomy.qza from Mike

Visualize taxonomy

`qiime metadata tabulate --m-input-file taxonomy.qza --o-visualization taxonomy.qzv`

Generate taxonomy bar plots

`qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv
`



.qzv files visualized at https://view.qiime2.org/

# 2. Shotgun sequencing

Download metagenome assembly

https://www.dropbox.com/s/f5j52tliumt6etm/G12_assembly.fna.gz?dl=0

Download Tannarella forsythia genome (fasta and gff3)

https://www.ncbi.nlm.nih.gov/nuccore/NC_016610.1

Align  metagenome contigs on the T. forsynthia genome

In [ ]:
! bwa index sequence.fasta

[bwa_index] Pack FASTA... 0.03 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 1.18 seconds elapse.
[bwa_index] Update BWT... 0.02 sec
[bwa_index] Pack forward-only FASTA... 0.02 sec
[bwa_index] Construct SA from BWT and Occ... 0.70 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index sequence.fasta
[main] Real time: 1.974 sec; CPU: 1.964 sec


In [ ]:
! bwa mem sequence.fasta G12_assembly.fna.gz | samtools view -b > alignment.bam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 33142 sequences (10000684 bp)...
[M::process] read 34258 sequences (10000241 bp)...
[M::mem_process_seqs] Processed 33142 reads in 6.873 CPU sec, 6.770 real sec
[M::process] read 35404 sequences (10000120 bp)...
[M::mem_process_seqs] Processed 34258 reads in 6.528 CPU sec, 6.401 real sec
[M::process] read 36534 sequences (10000152 bp)...
[M::mem_process_seqs] Processed 35404 reads in 6.549 CPU sec, 6.429 real sec
[M::process] read 38278 sequences (10001697 bp)...
[M::mem_process_seqs] Processed 36534 reads in 6.682 CPU sec, 6.559 real sec
[M::process] read 38866 sequences (10000030 bp)...
[M::mem_process_seqs] Processed 38278 reads in 7.033 CPU sec, 6.917 real sec
[M::process] read 40534 sequences (10000318 bp)...
[M::mem_process_seqs] Processed 38866 reads in 8.058 CPU sec, 8.005 real sec
[M::process] read 42182 sequences (10000459 bp)...
[M::mem_process_seqs] Processed 40534 reads in 7.075 CPU sec, 6.955 real sec
[M::pr

In [ ]:
! samtools flagstat alignment.bam

905742 + 0 in total (QC-passed reads + QC-failed reads)
905601 + 0 primary
0 + 0 secondary
141 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
16539 + 0 mapped (1.83% : N/A)
16398 + 0 primary mapped (1.81% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [ ]:
! samtools sort alignment.bam > alignment_sorted.bam
! samtools index alignment_sorted.bam

Obtain new regions with bedtools intersect

In [ ]:
! bedtools bamtobed -i alignment_sorted.bam > alignment.bed

In [ ]:
! bedtools intersect -v -b alignment.bed -a sequence.gff3 > intersect.txt

In [ ]:
! grep -o -P product.* intersect.txt | grep -v hypothetic # in modern bacteria, but not in ancient

product=helix-turn-helix domain-containing protein;protein_id=WP_014223573.1;transl_table=11
product=lanthionine synthetase LanC family protein;protein_id=WP_041590995.1;transl_table=11
product=TIGR04157 family glycosyltransferase;protein_id=WP_014223582.1;transl_table=11
product=class I lanthipeptide;protein_id=WP_041590503.1;transl_table=11
product=lanthionine synthetase C family protein;protein_id=WP_014223583.1;transl_table=11
product=lantibiotic dehydratase family protein;protein_id=WP_041590506.1;transl_table=11
product=thiopeptide-type bacteriocin biosynthesis protein;protein_id=WP_041590507.1;transl_table=11
product=four helix bundle protein;protein_id=WP_014223665.1;transl_table=11
product=IS1595-like element ISTfo1 family transposase;protein_id=WP_041590537.1;transl_table=11
product=tRNA-Gly
product=tRNA-Gly
product=histidinol phosphate phosphatase;protein_id=WP_041590544.1;transl_table=11
product=NVEALA domain-containing protein;protein_id=WP_014223806.1;transl_table=11
prod